In [1]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
df=pd.read_csv("/content/APEXFOOT.csv",parse_dates=['DATE'])
df.head()

#       DATE TRADING CODE   LTP*  ...    YCP  TRADE  VALUE (mn)  VOLUME
0  439 2018-05-08     APEXFOOT  331.7  ...  341.2    143       5.122  15,334
1  438 2018-06-08     APEXFOOT  331.2  ...  332.7     94       3.217   9,626
2  437 2018-07-08     APEXFOOT  327.1  ...  331.8    123       3.645  11,049
3  436 2018-08-08     APEXFOOT  325.1  ...  327.7    118       6.380  19,344
4  435 2018-09-08     APEXFOOT  322.0  ...  328.4    165       7.030  21,712

[5 rows x 12 columns]

In [2]:
# Sort DataFrame by date
df = df.sort_values('DATE')

# Double check the result
df.head()

#       DATE TRADING CODE   LTP*  ...    YCP  TRADE  VALUE (mn)  VOLUME
36  403 2018-01-10     APEXFOOT  296.5  ...  298.9    139       2.509   8,392
59  380 2018-01-11     APEXFOOT  269.9  ...  270.7     72       1.005   3,724
37  402 2018-02-10     APEXFOOT  294.4  ...  297.9    237       5.574  18,816
79  360 2018-02-12     APEXFOOT  273.0  ...  273.9    107       2.479   9,064
16  423 2018-03-09     APEXFOOT  319.7  ...  309.5    224       8.640  26,931

[5 rows x 12 columns]

In [3]:
df = df.set_index('DATE')
df.head(3)

# TRADING CODE   LTP*   HIGH  ...    YCP  TRADE  VALUE (mn)  VOLUME
DATE                                        ...                                  
2018-01-10  403     APEXFOOT  296.5  300.1  ...  298.9    139       2.509   8,392
2018-01-11  380     APEXFOOT  269.9  273.0  ...  270.7     72       1.005   3,724
2018-02-10  402     APEXFOOT  294.4  297.9  ...  297.9    237       5.574  18,816

[3 rows x 11 columns]

In [4]:
df['daily_ret'] = df['CLOSEP*'].pct_change()
df['daily_ret'] = df['daily_ret'].fillna(0.0)

In [5]:
df['volatility']= df['daily_ret'].rolling(window=21).std()
df['volatility'] = df['volatility'].fillna(0.0)

In [6]:
df['volatility'] = df['volatility']*100

In [7]:
import matplotlib.pyplot as plt 

In [8]:
df=df.reset_index()['volatility']

In [9]:
df.shape

(439,)

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df=scaler.fit_transform(np.array(df).reshape(-1,1))

In [11]:
##splitting dataset into train and test split where train=70 percent and test=30 percent
training_size=int(len(df)*0.70)
test_size=len(df)-training_size
train_data,test_data=df[0:training_size,:],df[training_size:len(df),:1]




In [12]:
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [],[]
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [13]:

# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [14]:
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
import xgboost as xgb
from sklearn.datasets import make_regression

In [15]:
model1 = RandomForestRegressor(n_estimators=1000)
model2 =AdaBoostRegressor(n_estimators=1000,learning_rate=0.01)
model3= GradientBoostingRegressor(n_estimators=1000,learning_rate=0.01)
model4 = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.01)



In [16]:
model1.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [17]:
model2.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=0.01, loss='linear',
                  n_estimators=1000, random_state=None)

In [18]:
model3.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.01, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=1000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [19]:
model4.fit(X_train, y_train)

[13:25:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [20]:

train_predict1=model1.predict(X_train)
test_predict1= model1.predict(X_test)

In [21]:

train_predict2=model2.predict(X_train)
test_predict2= model2.predict(X_test)

In [22]:

train_predict3=model3.predict(X_train)
test_predict3= model3.predict(X_test)

In [23]:

train_predict4=model4.predict(X_train)
test_predict4= model4.predict(X_test)

In [ ]:
ytest

array([0.18707429, 0.18639876, 0.13280455, 0.01121447, 0.01121447,
       0.01121447, 0.01121447, 0.01121447, 0.01121447, 0.01121447,
       0.01121447, 0.01121447, 0.01121447, 0.01245888, 0.08721499,
       0.12679627, 0.12679627, 0.12667225, 0.17544398, 0.21035892,
       0.21065095, 0.21065095, 0.23960185, 0.25003415, 0.25437913,
       0.2823053 , 0.29316926, 0.29705464, 0.29710798, 0.31643045,
       0.32388954])

In [24]:
Y_test = np.reshape(ytest,(1, ytest.size))

In [25]:
Y_test =scaler.inverse_transform(Y_test)

In [26]:
Y_test

array([[2.39999932, 2.39133286, 1.70376606, 0.14387177, 0.14387177,
        0.14387177, 0.14387177, 0.14387177, 0.14387177, 0.14387177,
        0.14387177, 0.14387177, 0.14387177, 0.15983648, 1.11889194,
        1.62668515, 1.62668515, 1.62509398, 2.25079261, 2.69872078,
        2.70246724, 2.70246724, 3.07388197, 3.20771914, 3.26346145,
        3.62172975, 3.76110493, 3.81095094, 3.81163522, 4.0595256 ,
        4.15521911]])

In [27]:
test_predict1

array([0.64903475, 0.64920647, 0.64933863, 0.64880056, 0.65056801,
       0.65309607, 0.65183969, 0.652142  , 0.65255543, 0.65165964,
       0.65385556, 0.65308484, 0.65315177, 0.6536055 , 0.650074  ,
       0.64847857, 0.64814579, 0.64787578, 0.64816683, 0.64796632,
       0.6463898 , 0.64725522, 0.64684459, 0.6461373 , 0.64582834,
       0.64512486, 0.64557045, 0.64300781, 0.6435606 , 0.64164196,
       0.6450761 ])

In [28]:
test_predict1 = np.reshape(test_predict1,(1, test_predict1.size))
test_predict2 = np.reshape(test_predict2,(1, test_predict2.size))
test_predict3 = np.reshape(test_predict3,(1, test_predict3.size))
test_predict4 = np.reshape(test_predict4,(1, test_predict3.size))



In [29]:
test_predict1 =scaler.inverse_transform(test_predict1)
test_predict2 =scaler.inverse_transform(test_predict2)
test_predict3 =scaler.inverse_transform(test_predict3)
test_predict4 =scaler.inverse_transform(test_predict4)

In [30]:
test_predict1

array([[8.32654751, 8.32875045, 8.33044595, 8.32354298, 8.34621791,
        8.37865068, 8.36253246, 8.36641076, 8.37171472, 8.36022248,
        8.38839425, 8.37850654, 8.3793652 , 8.38518624, 8.33988018,
        8.31941223, 8.31514288, 8.31167885, 8.31541278, 8.31284047,
        8.29261503, 8.30371762, 8.29844965, 8.28937577, 8.28541201,
        8.27638696, 8.28210355, 8.24922717, 8.25631888, 8.23170446,
        8.27576147]])

In [31]:
test_predict2

array([[9.16270649, 9.16270649, 9.16270649, 9.16270649, 9.16270649,
        9.16270649, 9.16270649, 9.16270649, 9.16270649, 9.16270649,
        9.16270649, 9.1397927 , 9.14094285, 9.12110774, 9.01434925,
        9.00597999, 9.01447027, 9.01434925, 9.01434925, 9.01434925,
        9.00597999, 9.01434925, 9.05045974, 9.03287329, 9.01485808,
        9.01485808, 9.02093547, 9.02093547, 9.02093547, 9.01434925,
        9.11788694]])

In [32]:
test_predict3

array([[9.41745545, 9.41397585, 9.42044603, 9.36733966, 9.37054677,
        9.29093774, 9.30254936, 9.32628344, 9.31677191, 9.36811196,
        9.36844213, 9.25383165, 9.29965402, 9.33496095, 9.51312589,
        9.52806474, 9.541701  , 9.52479676, 9.55003637, 9.42968308,
        9.4128476 , 9.43325408, 9.43325408, 9.20655669, 9.22215265,
        9.23171747, 9.23288469, 9.29789444, 9.42063374, 9.39746479,
        9.29411009]])

In [33]:
test_predict4

array([[9.350796, 9.36103 , 9.45151 , 9.469278, 9.466263, 9.38903 ,
        9.378258, 9.378258, 9.378258, 9.437958, 9.366757, 9.359419,
        9.359419, 9.401088, 8.949679, 8.921458, 8.930824, 8.966254,
        8.907263, 8.776982, 8.80237 , 8.768712, 8.779644, 8.721392,
        8.721392, 8.721392, 8.721392, 8.744588, 8.854827, 9.003213,
        8.983838]], dtype=float32)

In [34]:
#Y_test = Y_test.transpose()
RMSE1 = math.sqrt(mean_squared_error(Y_test,test_predict1))
RMSE1

6.646602886836347

In [35]:
#Y_test = Y_test.transpose()
RMSE2 = math.sqrt(mean_squared_error(Y_test,test_predict2))
RMSE2

7.39275082353764

In [36]:
#Y_test = Y_test.transpose()
RMSE3 = math.sqrt(mean_squared_error(Y_test,test_predict3))
RMSE3

7.6668212985814295

In [37]:
#Y_test = Y_test.transpose()
RMSE4 = math.sqrt(mean_squared_error(Y_test,test_predict4))
RMSE4

7.441609414888624

In [38]:

from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict1)
print('MAE: %f' % mae)

MAE: 6.476492


In [39]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict2)
print('MAE: %f' % mae)

MAE: 7.238297


In [40]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict3)
print('MAE: %f' % mae)

MAE: 7.526155


In [41]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict4)
print('MAE: %f' % mae)

MAE: 7.245544
